<a href="https://colab.research.google.com/github/aysegulguzel/Generative-AI-Engineering/blob/main/GenAI_RAG_with_Pinecone_Weaviate_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pinecone-client==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 582.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 3.4 MB/s eta 0:00:00


In [ ]:
import os

os.environ["PINECONE_API_KEY"] = "YOUR-API-HERE"
#os.environ["PINECONE_ENV"] = "<FILL IN>"

In [ ]:
import pinecone

pinecone_api_key = os.environ["PINECONE_API_KEY"]
#pinecone_env = os.environ["PINECONE_ENV"]

pinecone.init(api_key=pinecone_api_key) #, environment=pinecone_env)

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f4697f80b22269f8fec5c2cd14e5bf250be16933a2eb7c2471e37de8279bed6e
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

pdf = pd.read_csv(f"/content/drive/MyDrive/labelled_newscatcher_dataset.csv", sep=";")
pdf["id"] = pdf.index
display(pdf)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,0
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,1
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,2
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,3
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,4
...,...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en,108769
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en,108770
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en,108771
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en,108772


In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

# Create a SparkSession
spark = SparkSession.builder \
    .appName("PineconeRAG") \
    .getOrCreate()

# Assuming `pdf` is the path to your CSV file in Google Colab
# You can upload your CSV file to Google Colab and then specify its path here
pdf = "/content/drive/MyDrive/labelled_newscatcher_dataset.csv"

# Read the CSV file into a DataFrame and add a new column "id" with UUID values
df = spark.read.option("header", True) \
    .option("sep", ";") \
    .format("csv") \
    .load(pdf) \
    .withColumn("id", F.expr("uuid()"))

# Display the DataFrame
df.show()




+-------+--------------------+--------------------+-------------------+--------------------+----+--------------------+
|  topic|                link|              domain|     published_date|               title|lang|                  id|
+-------+--------------------+--------------------+-------------------+--------------------+----+--------------------+
|SCIENCE|https://www.eurek...|      eurekalert.org|2020-08-06 13:59:45|A closer look at ...|  en|e1bf8f3d-3c98-4d9...|
|SCIENCE|https://www.pulse...|            pulse.ng|2020-08-12 15:14:19|An irresistible s...|  en|ce45ebdb-1089-41d...|
|SCIENCE|https://www.expre...|       express.co.uk|2020-08-13 21:01:00|Artificial intell...|  en|629cfdff-99c0-4a3...|
|SCIENCE|https://www.ndtv....|            ndtv.com|2020-08-03 22:18:26|Glaciers Could Ha...|  en|ddd06775-bf2d-405...|
|SCIENCE|https://www.thesu...|           thesun.ie|2020-08-12 19:54:36|Perseid meteor sh...|  en|78ac4bc0-8ce5-45d...|
|SCIENCE|https://interesti...|interestingengine.

Generate the embeddings first and save them to a dataframe

Method 1: Upsert to Pinecone in batches

In [ ]:
#Upsert to Pinecone in batches
pdf = df.limit(1000).toPandas()
display(pdf)

,topic,link,domain,published_date,title,lang,id
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en,6402ee2f-ea10-4621-95aa-c5334e8c8a19
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en,39769c2a-a7f1-4bc5-81f1-c91866140b7c
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en,001322a7-4e8d-4e98-bdda-ebd94a8602f5
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en,2a207707-dffb-421e-8d2d-a08be7582831
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en,c2c4d562-46a5-4dec-95d1-d558e4c562ba
...,...,...,...,...,...,...,...
995,TECHNOLOGY,https://www.androidcentral.com/mate-40-will-be...,androidcentral.com,2020-08-07 17:12:33,The Mate 40 will be the last Huawei phone with...,en,3f7ea6b9-f913-4a5a-9138-501fd68d9716
996,SCIENCE,https://www.cnn.com/2020/08/17/africa/stone-ag...,cnn.com,2020-08-17 17:10:00,"Early humans knew how to make comfy, pest-free...",en,6f13ef7a-768c-4be9-a105-ef49aad8a3df
997,HEALTH,https://www.tenterfieldstar.com.au/story/68776...,tenterfieldstar.com.au,2020-08-13 03:26:06,Regional Vic set for virus testing blitz,en,74fa376a-c7dd-4ee8-93d6-afc9d59ddb07
998,HEALTH,https://news.sky.com/story/coronavirus-trials-...,news.sky.com,2020-08-13 13:22:58,Coronavirus: Trials of second contact-tracing ...,en,880f1a7d-b240-490e-9996-e16f53366307


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 36.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 707.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 503.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 939.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 841.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 890.7 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 719.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 798.9 kB/s eta 0:00:00
     ━━

In [ ]:
from sentence_transformers import SentenceTransformer

# Specify the cache folder path inside Google Colab
cache_folder_path = "/content/cache_folder/"

# We will use embeddings from this model to apply to our data
model = SentenceTransformer(
    "all-MiniLM-L6-v2", cache_folder=cache_folder_path
)  # Use a pre-cached model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
#delete the index, if it already exist. pinecone free version let you create just 1 index
pinecone_index_name = "news"

if pinecone_index_name in pinecone.list_indexes():
    pinecone.delete_index(pinecone_index_name)

In [ ]:
# Only create the index if it doesn't exist
if pinecone_index_name not in pinecone.list_indexes():
    try:
        pinecone.create_index(
            name=pinecone_index_name,
            dimension=model.get_sentence_embedding_dimension(),
            metric="cosine",
        )
        print(f"Index '{pinecone_index_name}' created successfully.")
    except pinecone.exceptions.ApiException as e:
        print(f"Failed to create index: {e}")
else:
    print(f"Index '{pinecone_index_name}' already exists.")

# Now connect to the index
pinecone_index = pinecone.Index(pinecone_index_name)


Failed to create index: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '9d9c0aac6f3e2c55e26a7177e2151fd8', 'Date': 'Fri, 22 Mar 2024 14:57:11 GMT', 'Server': 'Google Frontend', 'Content-Length': '72', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Capacity Reached. Increase your quota or upgrade to create more indexes.



In [ ]:
# only create index if it doesn't exist
if pinecone_index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=pinecone_index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric="cosine",
    )

# now connect to the index
pinecone_index = pinecone.Index(pinecone_index_name)

ApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '835a1fb8d8cc8abb079f814b72b96402', 'Date': 'Fri, 22 Mar 2024 14:57:04 GMT', 'Server': 'Google Frontend', 'Content-Length': '72', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: Capacity Reached. Increase your quota or upgrade to create more indexes.


In [ ]:
#write vectors into index (upsert)
from tqdm.auto import tqdm

batch_size = 1000

for i in tqdm(range(0, len(pdf["title"]), batch_size)):
    # find end of batch
    i_end = min(i + batch_size, len(pdf["title"]))
    # create IDs batch
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadata = [{"title": title} for title in pdf["title"][i:i_end]]
    # create embeddings
    embedding_title_batch = model.encode(pdf["title"][i:i_end]).tolist()
    # create records list for upsert
    records = zip(ids, embedding_title_batch, metadata)
    # upsert to Pinecone
    pinecone_index.upsert(vectors=records)

# check number of records in the index
pinecone_index.describe_index_stats()

  0%|          | 0/1 [00:00<?, ?it/s]

PineconeProtocolError: Failed to connect; did you specify the correct index name?

In [ ]:
#query the index directly
query = "fish"

# create the query vector
user_query = model.encode(query).tolist()

# submit the query to the Pinecone index
pinecone_answer = pinecone_index.query(user_query, top_k=3, include_metadata=True)

for result in pinecone_answer["matches"]:
    print(f"{round(result['score'], 2)}, {result['metadata']['title']}")
    print("-" * 120)

Method 2: Process with Spark and write to Pinecone with Spark

In [ ]:
import pandas as pd
from pyspark.sql.functions import pandas_udf
from sentence_transformers import SentenceTransformer
from typing import Iterator

@pandas_udf("array<float>")
def create_embeddings_with_transformers(
    sentences: Iterator[pd.Series],) -> Iterator[pd.Series]:
    model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")
    for batch in sentences:
        yield pd.Series(model.encode(batch).tolist())

In [ ]:
import pyspark.sql.functions as F

transformer_type = "sentence-transformers/all-MiniLM-L6-v2"
embedding_spark_df = (
    df.limit(1000)
    .withColumn("values", create_embeddings_with_transformers("title"))
    .withColumn("namespace", F.lit(transformer_type))
    .withColumn("metadata", F.to_json(F.struct(F.col("topic").alias("TOPIC"))))
    # We select these columns because they are expected by the Spark-Pinecone connector
    .select("id", "values", "namespace", "metadata")
)
display(embedding_spark_df)

In [ ]:
pinecone_index_name = "news"

if pinecone_index_name in pinecone.list_indexes():
    pinecone.delete_index(pinecone_index_name)

# only create index if it doesn't exist
model = SentenceTransformer(transformer_type)
if pinecone_index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=pinecone_index_name,
        dimension=model.get_sentence_embedding_dimension(),
        metric="cosine",
    )

# now connect to the index
pinecone_index = pinecone.Index(pinecone_index_name)

RAG with Weaviate

In [ ]:
pip install weaviate-client==3.19.1

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "FILL-HERE"
os.environ["WEAVIATE_NETWORK"] = "FILL-HERE"

In [ ]:
!pip install openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [ ]:
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]
weaviate_network = os.environ["WEAVIATE_NETWORK"]

In [ ]:
import weaviate

client = weaviate.Client(
    weaviate_network, additional_headers={"X-OpenAI-Api-Key": openai.api_key}
)
client.is_ready()

True

In [ ]:
display(df)

DataFrame[topic: string, link: string, domain: string, published_date: string, title: string, lang: string, id: string]

In [ ]:
#define a schema
class_name = "News"
class_obj = {
    "class": class_name,
    "description": "News topics collected by NewsCatcher",
    "properties": [
        {"name": "topic", "dataType": ["string"]},
        {"name": "link", "dataType": ["string"]},
        {"name": "domain", "dataType": ["string"]},
        {"name": "published_date", "dataType": ["string"]},
        {"name": "title", "dataType": ["string"]},
        {"name": "lang", "dataType": ["string"]},
    ],
    "vectorizer": "text2vec-openai",
}

In [ ]:
# If the class exists before, we will delete it first
if client.schema.exists(class_name):
    print("Deleting existing class...")
    client.schema.delete_class(class_name)

print(f"Creating class: '{class_name}'")
client.schema.create_class(class_obj)

Creating class: 'News'


In [ ]:
import json

print(json.dumps(client.schema.get(class_name), indent=4))

{
    "class": "News",
    "description": "News topics collected by NewsCatcher",
    "invertedIndexConfig": {
        "bm25": {
            "b": 0.75,
            "k1": 1.2
        },
        "cleanupIntervalSeconds": 60,
        "stopwords": {
            "additions": null,
            "preset": "en",
            "removals": null
        }
    },
    "moduleConfig": {
        "text2vec-openai": {
            "baseURL": "https://api.openai.com",
            "model": "ada",
            "vectorizeClassName": true
        }
    },
    "multiTenancyConfig": {
        "enabled": false
    },
    "properties": [
        {
            "dataType": [
                "text"
            ],
            "indexFilterable": true,
            "indexSearchable": true,
            "moduleConfig": {
                "text2vec-openai": {
                    "skip": false,
                    "vectorizePropertyName": false
                }
            },
            "name": "topic",
            "tokenizat

In [ ]:
#https://github.com/weaviate/spark-connector/releases/tag/v1.3.2

SyntaxError: invalid syntax (<ipython-input-15-a3f024a26aa8>, line 1)

In [ ]:
(
    df.limit(100)
    .write.format("io.weaviate.spark.Weaviate")
    .option("scheme", "http")
    .option("host", weaviate_network.split("https://")[1])
    .option("header:X-OpenAI-Api-Key", openai.api_key)
    .option("className", class_name)
    .mode("append")
    .save()
)

Py4JJavaError: An error occurred while calling o44.save.
: org.apache.spark.SparkClassNotFoundException: [DATA_SOURCE_NOT_FOUND] Failed to find the data source: io.weaviate.spark.Weaviate. Please find packages at `https://spark.apache.org/third-party-projects.html`.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.dataSourceNotFoundError(QueryExecutionErrors.scala:724)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSource(DataSource.scala:647)
	at org.apache.spark.sql.execution.datasources.DataSource$.lookupDataSourceV2(DataSource.scala:697)
	at org.apache.spark.sql.DataFrameWriter.lookupV2Provider(DataFrameWriter.scala:863)
	at org.apache.spark.sql.DataFrameWriter.saveInternal(DataFrameWriter.scala:257)
	at org.apache.spark.sql.DataFrameWriter.save(DataFrameWriter.scala:248)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	a

In [ ]:
client.query.get("News", ["topic"]).do()

In [ ]:
where_filter = {
    "path": ["topic"],
    "operator": "Equal",
    "valueString": "SCIENCE",
}

# We are going to search for any titles related to locusts
near_text = {"concepts": "locust"}
(
    client.query.get(class_name, ["topic", "domain", "title"])
    .with_where(where_filter)
    .with_near_text(near_text)
    .with_limit(2)
    .do()
)

In [ ]:
import openai

model = "text-embedding-ada-002"
openai_object = openai.Embedding.create(input=["locusts"], model=model)

openai_embedding = openai_object["data"][0]["embedding"]

(
    client.query.get("News", ["topic", "domain", "title"])
    .with_where(where_filter)
    .with_near_vector(
        {
            "vector": openai_embedding,
            "distance": 0.7,  # this sets a threshold for distance metric
        }
    )
    .with_limit(2)
    .do()
)